# Classification Experiments on Motor Control Data

Motor data is described within README.

In [1]:
cd ../../../../

/Users/adam2392/Documents/SPORF


In [13]:
import pandas as pd
from scipy import stats
import numpy as np
import mne
import mne_bids
from pathlib import Path
from mne_bids import make_bids_basename

from natsort import natsorted
from pprint import pprint
    
# plotting
import matplotlib
import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap
%matplotlib inline
import seaborn as sns

import pickle
import time
from tqdm import tqdm

from sklearn.model_selection import train_test_split

# classifiers
from sklearn.linear_model import LogisticRegression
from sklearn.svm import LinearSVC
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.neighbors import KNeighborsClassifier
from statsmodels.tsa.vector_ar.var_model import VARProcess
# from rerf.rerfClassifier import rerfClassifier

import sys
print(sys.version)

from mtsmorf.io.read import get_trial_info, \
    read_dataset, read_label, read_raw_bids

import warnings
warnings.simplefilter('ignore')

3.8.2 | packaged by conda-forge | (default, Mar  5 2020, 16:54:44) 
[Clang 9.0.1 ]


# Load in the Data

In [19]:
def load_war_data(bids_root):
    subjects = [x.name for x in Path(bids_root).glob("*") if x.is_dir()]
    session = "efri"
    task = "war"
    acquisition = "seeg"
    kind = "ieeg"

    for subject in subjects:
        # get subject directory
        subj_dir = Path(Path(bids_root) / subject)
        bids_fnames = natsorted(
                [fpath.name for fpath in subj_dir.rglob(f"*_{kind}.vhdr") 
                 if all(x in fpath.name for x in [session, task, acquisition, kind])]
            )
        
        print(bids_fnames)
        
        # for each file
        for bids_fname in bids_fnames:
            pass
        
load_war_data(bids_root)

[]
[]
['sub-efri18_ses-efri_task-war_acq-seeg_run-01_ieeg.vhdr']
[]
['sub-efri16_ses-efri_task-war_acq-seeg_run-01_ieeg.vhdr']
['sub-efri17_ses-efri_task-war_acq-seeg_run-01_ieeg.vhdr']
['sub-efri10_ses-efri_task-war_acq-seeg_run-01_ieeg.vhdr']
[]
['sub-efri21_ses-efri_task-war_acq-seeg_run-01_ieeg.vhdr']
['sub-efri07_ses-efri_task-war_acq-seeg_run-01_ieeg.vhdr']
['sub-efri09_ses-efri_task-war_acq-seeg_run-01_ieeg.vhdr']
['sub-efri06_ses-efri_task-war_acq-seeg_run-01_ieeg.vhdr']
['sub-efri15_ses-efri_task-war_acq-seeg_run-01_ieeg.vhdr']
['sub-efri12_ses-efri_task-war_acq-seeg_run-01_ieeg.vhdr']
['sub-efri13_ses-efri_task-war_acq-seeg_run-01_ieeg.vhdr']
['sub-efri14_ses-efri_task-war_acq-seeg_run-01_ieeg.vhdr']
[]


In [9]:
bids_root = "/Users/adam2392/Dropbox/efri/"

# subject identifiers
subject = "efri06"
session = "efri"
task = "war"
acquisition = "seeg"
run = "01"

kind = "ieeg"

# if we want to look at a specific id
trial_id = 2

# bids filename
bids_fname = make_bids_basename(
    subject=subject,
    session=session,
    task=task,
    acquisition=acquisition,
    run=run,
    suffix=f"{kind}.vhdr",
)

pprint(f"Bids root is: {bids_root}")
pprint(f"Session experimental filename: {bids_fname}")

'Bids root is: /Users/adam2392/Dropbox/efri/'
('Session experimental filename: '
 'sub-efri06_ses-efri_task-war_acq-seeg_run-01_ieeg.vhdr')


In [ ]:
# read a specific trial
rawdata, times, events_tsv = read_trial(bids_fname, bids_root, trial_id)

# get the label of this trial
y, trial_ids = read_label(
    bids_fname, bids_root, trial_id=None, label_keyword="bet_amount"
)
unsuccessful_trial_inds = np.where(np.isnan(y))[
    0
]  # get unsuccessful trials based on keyword label

# read dataset as an epoch
epochs = read_dataset(bids_fname, bids_root)
epochs = epochs.drop(unsuccessful_trial_inds)
epochs.load_data()
epochs_data = epochs.get_data()

print("Selected events in events.tsv: ", epochs.selection)
print("Number of epochs: ", len(epochs))
print("Data shape: ", epochs_data.shape)

print(len(y))
print("Unique labels (e.g. bet amounts): ", np.unique(y))

print("Specific trial: ", rawdata.shape)
print("Events data structure for specific trial: ", events_tsv)
print(bids_fname)
print(times[0:5])
